My plan is to first pre-allocate slots and gates, considering the strong correlation between slots and gates (especially bridge-side gates). Pre-allocate slots based on constraints such as aircraft size, aircraft parking and turnaround time, slot allocation zones (for example, some airlines have fixed slot allocation zones), and prioritize flights with short turnaround times. Then, based on the correspondence between slots and gates, pre-allocate gates, while minimizing passenger walking distances. This pre-allocation phase incorporates GCN for multi-objective optimization.

Based on this pre-allocation, dynamic allocation is performed, dynamically adjusting slots and gates based on existing constraints to accommodate delayed flights.

# Pre-allocation

At first, I use this Python script simulates a simple gate allocation process at an airport. It:

- Creates gate and flight data.

- Attempts to assign each flight to a gate at the same terminal, while avoiding time conflicts.

- Outputs the assignment result as a pandas DataFrame.

In [2]:
import random
import pandas as pd
from datetime import datetime, timedelta

# Set random seed for reproducibility
random.seed(42)

# Simulate gate information (5 gates per terminal)
gates = []
for terminal in ['T1', 'T2']:
    for i in range(1, 6):
        gates.append({'gate_id': f'{terminal}-G{i}', 'terminal': terminal})

# Simulate 10 flights with arrival, departure times and terminal info
flights = []
base_time = datetime(2025, 8, 1, 6, 0)
for i in range(10):
    arrival = base_time + timedelta(minutes=random.randint(0, 300))
    departure = arrival + timedelta(minutes=random.randint(45, 90))
    terminal = random.choice(['T1', 'T2'])
    flights.append({
        'flight_id': f'F{i+1:03}',
        'arrival': arrival,
        'departure': departure,
        'terminal': terminal
    })

# Initialize gate usage records: {gate_id: [(start_time, end_time), ...]}
gate_schedule = {g['gate_id']: [] for g in gates}

# Gate assignment function
def assign_gate(flight):
    terminal = flight['terminal']
    for gate in [g for g in gates if g['terminal'] == terminal]:
        occupied = gate_schedule[gate['gate_id']]
        # Check for time conflict with existing usage
        conflict = any(not (flight['departure'] <= s or flight['arrival'] >= e) for s, e in occupied)
        if not conflict:
            gate_schedule[gate['gate_id']].append((flight['arrival'], flight['departure']))
            return gate['gate_id']
    return None  # No gate available

# Assign gates to all flights
for flight in flights:
    flight['assigned_gate'] = assign_gate(flight)

# Output the assignment result
df_result = pd.DataFrame(flights)
print(df_result[['flight_id', 'terminal', 'arrival', 'departure', 'assigned_gate']])


  flight_id terminal             arrival           departure assigned_gate
0      F001       T2 2025-08-01 06:57:00 2025-08-01 07:43:00         T2-G1
1      F002       T1 2025-08-01 08:05:00 2025-08-01 09:04:00         T1-G1
2      F003       T1 2025-08-01 06:52:00 2025-08-01 08:20:00         T1-G2
3      F004       T1 2025-08-01 09:36:00 2025-08-01 10:23:00         T1-G1
4      F005       T1 2025-08-01 06:47:00 2025-08-01 07:45:00         T1-G1
5      F006       T1 2025-08-01 10:18:00 2025-08-01 11:41:00         T1-G2
6      F007       T2 2025-08-01 10:47:00 2025-08-01 11:44:00         T2-G1
7      F008       T2 2025-08-01 07:52:00 2025-08-01 09:05:00         T2-G1
8      F009       T2 2025-08-01 06:03:00 2025-08-01 06:58:00         T2-G2
9      F010       T1 2025-08-01 08:54:00 2025-08-01 09:56:00         T1-G2


Gates: Two terminals (T1 and T2), each with 5 gates.

Flights: 10 flights randomly scheduled with variable arrival/departure and assigned a terminal.

Assignment: For each flight, it tries to assign a gate without overlapping with existing assignments.

Conflict check: not (flight['departure'] <= s or flight['arrival'] >= e) ensures time intervals don’t overlap.

In [3]:
# Simplified flight generation
import pandas as pd
import numpy as np

n_flights = 100
airlines = ['KLM', 'EasyJet', 'Transavia']
aircraft_types = ['A320', 'B737', 'E190']

# Generate a DataFrame of flights with airline, aircraft type, and scheduled arrival times
flights = pd.DataFrame({
    'flight_id': [f'KL{1000+i}' for i in range(n_flights)],
    'airline': np.random.choice(airlines, n_flights),
    'aircraft_type': np.random.choice(aircraft_types, n_flights),
    'scheduled_arrival': pd.date_range('2025-08-01 06:00', periods=n_flights, freq='7min'),
})

# Simulate random delays (in minutes)
flights['delay'] = np.random.choice([0, 5, 10, 20], n_flights, p=[0.6, 0.2, 0.15, 0.05])

# Compute actual arrival time by adding the delay
flights['actual_arrival'] = flights['scheduled_arrival'] + pd.to_timedelta(flights['delay'], unit='min')

# Display the first few rows
flights.head()


,flight_id,airline,aircraft_type,scheduled_arrival,delay,actual_arrival
0,KL1000,EasyJet,E190,2025-08-01 06:00:00,10,2025-08-01 06:10:00
1,KL1001,Transavia,E190,2025-08-01 06:07:00,0,2025-08-01 06:07:00
2,KL1002,Transavia,E190,2025-08-01 06:14:00,0,2025-08-01 06:14:00
3,KL1003,Transavia,B737,2025-08-01 06:21:00,10,2025-08-01 06:31:00
4,KL1004,KLM,E190,2025-08-01 06:28:00,0,2025-08-01 06:28:00


This code simulates a basic flight schedule generator, useful for creating input data for models like GNN (Graph Neural Networks) or DRL (Deep Reinforcement Learning).

- Define constants:

n_flights: Number of flights to simulate.

airlines: A list of airline names.

aircraft_types: Types of aircraft.

- Generate flight data:

Each flight gets a unique ID like KL1000, KL1001, etc.

airline and aircraft_type are randomly chosen for each flight.

scheduled_arrival is a regular 7-minute interval starting from 2025-08-01 06:00.

- Simulate flight delays:

Each flight randomly receives a delay of 0, 5, 10, or 20 minutes.

The probability distribution is:

60% chance: no delay,

20%: 5 minutes,

15%: 10 minutes,

5%: 20 minutes.

- Compute actual arrival time:

Add the delay to the scheduled arrival using pd.to_timedelta.



In [4]:
import random
from datetime import datetime, timedelta

# Flight class: represents a single flight
class Flight:
    def __init__(self, fid, arrival, departure, aircraft_type, airline):
        self.fid = fid  # flight ID
        self.arrival = arrival  # arrival time
        self.departure = departure  # departure time
        self.aircraft_type = aircraft_type  # type of aircraft
        self.airline = airline  # airline name
        self.assigned_stand = None  # assigned stand (initially none)
        self.assigned_gate = None  # assigned gate (initially none)

# Stand class: represents a parking stand for aircraft
class Stand:
    def __init__(self, sid, compatible_types, zone, has_bridge):
        self.sid = sid  # stand ID
        self.compatible_types = compatible_types  # list of compatible aircraft types
        self.zone = zone  # location zone (e.g., North)
        self.has_bridge = has_bridge  # whether the stand has a boarding bridge
        self.occupied_until = None  # time until the stand is occupied

# Gate class: represents a boarding gate
class Gate:
    def __init__(self, gid, terminal, linked_stands):
        self.gid = gid  # gate ID
        self.terminal = terminal  # terminal name
        self.linked_stands = linked_stands  # list of stands connected to this gate
        self.occupied_until = None  # time until the gate is occupied

# Initialize example flights (10 flights with increasing times)
flights = [
    Flight(f"KL{100+i}", datetime(2025,8,1,6+i,0), datetime(2025,8,1,7+i,0), "A320", "KLM")
    for i in range(10)
]

# Initialize stands: 6 stands compatible with A320 and B737, every second one has a bridge
stands = [
    Stand(f"S{i}", ["A320", "B737"], zone="North", has_bridge=(i%2==0))
    for i in range(6)
]

# Initialize gates: 5 gates each linked to one or two stands
gates = [
    Gate(f"G{i}", terminal="T1", linked_stands=[f"S{i}", f"S{i+1}"] if i+1 < 6 else [f"S{i}"])
    for i in range(5)
]

# Improved allocation function: assigns flights to stands and gates
def improved_allocator(flights, stands, gates):
    buffer = timedelta(minutes=15)  # turnaround buffer time between uses

    for flight in flights:
        # Filter compatible and available stands (based on type and buffer time)
        candidate_stands = [
            s for s in stands
            if flight.aircraft_type in s.compatible_types
            and (s.occupied_until is None or s.occupied_until + buffer <= flight.arrival)
        ]
        # Sort stands: prioritize North zone and stands with bridges
        candidate_stands.sort(key=lambda s: (s.zone != "North", not s.has_bridge))

        for stand in candidate_stands:
            # Try to find an available gate linked to this stand
            for gate in gates:
                if stand.sid in gate.linked_stands and (gate.occupied_until is None or gate.occupied_until + buffer <= flight.arrival):
                    # Assign the flight
                    flight.assigned_stand = stand.sid
                    flight.assigned_gate = gate.gid
                    stand.occupied_until = flight.departure
                    gate.occupied_until = flight.departure
                    break
            if flight.assigned_stand:
                break  # assignment complete for this flight

# Run allocation
improved_allocator(flights, stands, gates)

# Print the assignment results
for f in flights:
    print(f"Flight {f.fid} → Stand: {f.assigned_stand}, Gate: {f.assigned_gate}")


Flight KL100 → Stand: S0, Gate: G0
Flight KL101 → Stand: S2, Gate: G1
Flight KL102 → Stand: S0, Gate: G0
Flight KL103 → Stand: S2, Gate: G1
Flight KL104 → Stand: S0, Gate: G0
Flight KL105 → Stand: S2, Gate: G1
Flight KL106 → Stand: S0, Gate: G0
Flight KL107 → Stand: S2, Gate: G1
Flight KL108 → Stand: S0, Gate: G0
Flight KL109 → Stand: S2, Gate: G1


In the gate and stand allocation process, several operational constraints are considered to ensure feasible and conflict-free assignments.

First, each flight can only be assigned to a stand that is compatible with its aircraft type. If flight $i$ is assigned to stand $j$, then the aircraft type of flight $i$ must be among the types supported by stand $j$:

$$
\text{aircraft_type}_i\in \text{compatible_types}_j
$$

Secondly, a time conflict constraint is applied to avoid overlapping usage of stands. Let $A_i$ be the arrival time of flight $i$, $D_i$ be the departure time, and $B$ be a predefined buffer time between successive flights. The stand $j$ is available for flight $i$ only if its last occupied time plus buffer is earlier than $A_i$:

$$
O_j + B \leq A_i
$$

In addition, each gate is only connected to a subset of stands. If a flight is assigned both a stand $j$ and gate $k$, then the stand must be physically linked to the gate:

$$
\text{stand}_j \in \text{linked_stands}_k
$$

Lastly, assume that airlines prefer to assign flights to stands located in the "North" zone and those that have boarding bridges. This is implemented by sorting candidate stands with the following logic:

$$
\text{Sort by } (\text{zone}_j \ne \text{"North"},\ \neg \text{has_bridge}_j)
$$


In [5]:
# Install a compatible PyTorch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install PyG and its dependencies
!pip install torch-geometric -f https://data.pyg.org/whl/torch-2.0.0+cu118.html


Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6.8 MB/s e

In [6]:
# Check the installation
import torch
import torch_geometric
print(torch.__version__)
print(torch_geometric.__version__)

2.6.0+cu118
2.6.1


In [8]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# 1. Simple encoding functions for categorical data
def encode_aircraft(aircraft_type):
    return {"A320": 0, "B737": 1}.get(aircraft_type, -1)

def encode_airline(airline):
    return {"KLM": 0}.get(airline, 0)

def encode_terminal(terminal):
    return {"T1": 0}.get(terminal, 0)

# 2. Build graph structure: nodes and edges
flight_features = []
gate_features = []
edge_index = [[], []]  # Coordinate (COO) format edge list

flight_id_to_idx = {}
gate_id_to_idx = {}

for i, f in enumerate(flights):
    flight_id_to_idx[f.fid] = i
    t1 = (f.arrival.hour - 6) / 12  # Normalize time (06:00–18:00 → [0, 1])
    t2 = (f.departure.hour - 6) / 12
    flight_features.append([
        t1, t2,
        encode_aircraft(f.aircraft_type),
        encode_airline(f.airline)
    ])

for j, g in enumerate(gates):
    idx = len(flights) + j
    gate_id_to_idx[g.gid] = idx
    gate_feat = [
        len(g.linked_stands),      # Number of connected stands
        encode_terminal(g.terminal),   # Encoded terminal
        0,                # Padding feature (for dimension alignment)
        0                # Padding feature
    ]
    gate_features.append(gate_feat)


    # Add edges between compatible flights and gates
    for f in flights:
        for s in stands:
            if s.sid in g.linked_stands and s.occupied_until is None:
                if f.aircraft_type in s.compatible_types:
                    flight_idx = flight_id_to_idx[f.fid]
                    edge_index[0].append(flight_idx)  # From flight
                    edge_index[1].append(idx)         # To gate


x = torch.tensor(flight_features + gate_features, dtype=torch.float)
edge_index = torch.tensor(edge_index, dtype=torch.long)

# 3. Define the GCN model
class GCNGateAssigner(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)  # Output is a score

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x.squeeze()  # Return score per node

# 4. Wrap features and edge list into a graph data object
data = Data(x=x, edge_index=edge_index)

# 5. Initialize model and perform forward pass (no training)
model = GCNGateAssigner(in_channels=x.shape[1], hidden_channels=8)
output_scores = model(data.x, data.edge_index)

# Print the scores of the first few edges (flight → gate)
print("Top edge scores:")
for i in range(5):
    src, dst = edge_index[0][i].item(), edge_index[1][i].item()
    print(f"Flight {flights[src].fid} → Gate {gates[dst - len(flights)].gid} Score: {output_scores[dst].item():.4f}")

Top edge scores:
Flight KL100 → Gate G0 Score: -1.4149
Flight KL101 → Gate G0 Score: -1.4149
Flight KL102 → Gate G0 Score: -1.4149
Flight KL103 → Gate G0 Score: -1.4149
Flight KL104 → Gate G0 Score: -1.4149


## GCN-based Airport Gate Assignment Explanation

This code builds a minimal **Graph Convolutional Network (GCN)** model to score the compatibility between incoming flights and available gates at an airport. The pipeline includes data encoding, graph construction, model definition, and inference.

### 1. Node Feature Encoding

I create two types of nodes:

- **Flight nodes**: Each flight is encoded with:
  - Normalized arrival and departure time:
    $[
    t_1 = \frac{\text{arrival\_hour} - 6}{12}, \quad t_2 = \frac{\text{departure\_hour} - 6}{12}
    ]$
  - Encoded aircraft type and airline.

- **Gate nodes**: Each gate includes:
  - Number of connected stands
  - Terminal (encoded)
  - Two padding zeros for dimensional alignment with flight features.

The resulting node feature matrix is:
$[
\mathbf{X} \in \mathbb{R}^{(N_f + N_g) \times d}
$]
where:
- $ N_f $: number of flights,
- $ N_g $: number of gates,
- $ d $: feature dimension (here d = 4).

### 2. Graph Construction

Edges are added from each flight node to each compatible gate node if:

- The gate links to a stand,
- The stand is not occupied,
- The stand is compatible with the aircraft type.

The graph is encoded using an edge list:
$[
\text{edge_index} \in \mathbb{N}^{2 \times E}
$]
in Coordinate (COO) format, where each column $ (i, j) $ represents an edge from node $( i \to j $).

### 3. GCN Model

The model is a 2-layer Graph Convolutional Network:

Layer 1: Applies graph convolution followed by ReLU activation

$\
\mathbf{H}^{(1)} = \text{ReLU}\left(\text{GCNConv}^{(1)}(\mathbf{X}, \mathbf{A})\right)
\$

Layer 2: Projects to a single scalar per node

$\
\mathbf{H}^{(2)} = \text{GCNConv}^{(2)}(\mathbf{H}^{(1)}, \mathbf{A})
\$

$\
\hat{\mathbf{y}} = \text{squeeze}(\mathbf{H}^{(2)})
\$


- $\mathbf{X} $: input node features,
- $\mathbf{A} $: adjacency matrix (implicitly encoded by `edge_index`),
- $ \hat{\mathbf{y}} $: scalar score for each node (used to assess compatibility).

Only the **gate nodes' scores** are meaningful for assignment. Flights do not receive outputs—they initiate the scoring edges.

### 4. Inference

The model is evaluated without training. Output scores are printed for the first few **flight → gate** edges.

Example:
```text
Flight KL123 → Gate G5 Score: 0.4782
```

This score can be interpreted as a **learned compatibility score** between a flight and a gate, which could later be used in optimization (e.g., greedy assignment or max matching).

Since the current GCN model only initializes model parameters but does not undergo training, this means:

- The model weights are randomly initialized.
- The output score is simply a forward pass based on the unlearned parameters.
- There is no loss function, objective optimization, or gradient updates.

As a result, the output values of all gate nodes are almost identical.


In [12]:
from collections import defaultdict

# Create a dictionary to store gate scores for each flight.
# Key: flight node index, Value: list of (gate node index, predicted score) tuples
flight_to_scores = defaultdict(list)

# Iterate through all edges in the graph (each edge is a possible assignment)
for i in range(edge_index.shape[1]):
    src = edge_index[0][i].item()  # Source node index (flight)
    dst = edge_index[1][i].item()  # Destination node index (gate)
    score = output_scores[dst].item()  # Model-predicted score for this gate node
    flight_to_scores[src].append((dst, score))  # Save the gate and its score

# For each flight, choose the gate with the highest predicted score
assignments = {}  # Dictionary to store final assignment: flight ID → gate ID
for f_idx, gate_scores in flight_to_scores.items():
    best_gate_idx, best_score = max(gate_scores, key=lambda x: x[1])  # Highest score
    flight = flights[f_idx]  # Get the flight object
    gate = gates[best_gate_idx - len(flights)]  # Map gate node index back to gate object
    assignments[flight.fid] = gate.gid  # Assign gate ID to flight ID

# Print the final gate assignment results
print("Gate Assignments:")
for fid, gid in assignments.items():
    print(f"Flight {fid} → Gate {gid}")


Gate Assignments:
Flight KL100 → Gate G0
Flight KL101 → Gate G0
Flight KL102 → Gate G0
Flight KL103 → Gate G0
Flight KL104 → Gate G0
Flight KL105 → Gate G0
Flight KL106 → Gate G0
Flight KL107 → Gate G0
Flight KL108 → Gate G0
Flight KL109 → Gate G0


This code assigns gates to flights based on predicted scores using a bipartite graph. Each edge from a flight node to a gate node represents a possible assignment, and its corresponding score indicates the suitability predicted by a machine learning model.

The algorithm proceeds in two steps:

1. **Score Aggregation:**  
   For each edge $e_i = (f_j, g_k)$ in the graph (represented by `edge_index`), where $f_j$ is a flight node and $g_k$ is a gate node, I record the predicted score $s_{jk} = \text{output_scores}[g_k]$ for assigning gate $g_k$ to flight $f_j$.

2. **Gate Assignment:**  
   For each flight $f_j$, I choose the gate $g_k$ with the highest score:
   $$
   g_k^* = \arg\max_{g_k} s_{jk}
   $$
   The selected gate $g_k^*$ is then assigned to flight $f_j$.

The final output is a mapping from flight IDs to gate IDs, printed in the format:  
`Flight {fid} → Gate {gid}`.


# Dynamic Allocation


In [11]:
import random
from datetime import timedelta
import pandas as pd
from collections import defaultdict
import torch

# GNN-based dynamic gate adjustment function
def dynamic_gate_adjustment_gnn(flights, gates, stands, assignments, model, edge_index, x, delay_threshold=10):
    buffer = timedelta(minutes=15)
    gate_map = {g.gid: g for g in gates}
    gate_idx_map = {g.gid: idx + len(flights) for idx, g in enumerate(gates)}  # map gate ID to node index

    # Recompute scores using GNN (updated flight times will be re-evaluated)
    model.eval()
    with torch.no_grad():
        scores = model(x, edge_index)

    # Process flights in chronological order (simulate time progression)
    for flight in sorted(flights, key=lambda f: f.arrival):
        fid = flight.fid
        original_gate_id = assignments.get(fid)

        # Simulate a random delay
        delay_minutes = random.choice([0, 0, 5, 10, 15])
        flight.arrival += timedelta(minutes=delay_minutes)
        flight.departure += timedelta(minutes=delay_minutes)

        if delay_minutes >= delay_threshold:
            # Get candidate gates that are time-compatible and aircraft-compatible
            compatible_gates = [
                g for g in gates
                if (g.occupied_until is None or g.occupied_until + buffer <= flight.arrival)
                and any(s.sid in g.linked_stands for s in stands if flight.aircraft_type in s.compatible_types)
            ]

            if compatible_gates:
                # Sort candidate gates based on GNN-predicted score
                flight_node_idx = [i for i, f_obj in enumerate(flights) if f_obj.fid == fid][0]
                gate_scores = []

                for g in compatible_gates:
                    gate_node_idx = gate_idx_map[g.gid]
                    score = scores[gate_node_idx].item()
                    gate_scores.append((g, score))

                # Sort by predicted score descending
                gate_scores.sort(key=lambda x: x[1], reverse=True)

                reassigned = False
                for g_obj, _ in gate_scores:
                    if g_obj.occupied_until is None or g_obj.occupied_until + buffer <= flight.arrival:
                        flight.assigned_gate = g_obj.gid
                        g_obj.occupied_until = flight.departure
                        assignments[fid] = g_obj.gid
                        reassigned = True
                        break

                if not reassigned:
                    flight.assigned_gate = original_gate_id  # fallback to original gate
            else:
                flight.assigned_gate = original_gate_id
        else:
            flight.assigned_gate = original_gate_id  # delay is minor, keep original assignment

    return assignments

# Execute GNN-based dynamic reassignment
adjusted_assignments = dynamic_gate_adjustment_gnn(
    flights, gates, stands, assignments, model, edge_index, x
)

# Show results
adjusted_results = [
    (f.fid, f.assigned_gate, f.arrival.strftime("%H:%M"), f.departure.strftime("%H:%M"))
    for f in flights
]
df_adjusted = pd.DataFrame(adjusted_results, columns=["Flight", "Gate", "Arrival", "Departure"])
df_adjusted


,Flight,Gate,Arrival,Departure
0,KL100,G0,06:00,07:00
1,KL101,G0,07:05,08:05
2,KL102,G0,08:00,09:00
3,KL103,G0,09:00,10:00
4,KL104,G2,10:10,11:10
5,KL105,G0,11:00,12:00
6,KL106,G0,12:05,13:05
7,KL107,G0,13:05,14:05
8,KL108,G2,14:15,15:15
9,KL109,G0,15:05,16:05


## Explanation of the Dynamic Gate Reallocation

This code performs dynamic gate reallocation using a simplified Graph Neural Network (GNN) logic. The main steps and concepts are explained below:

### 1. **Graph Construction**

I define a graph $G = (V, E)$ where:

- $V$ is the set of **flights** and **gates**.
- $E$ is the set of **edges** connecting flights to their candidate gates.

Each flight node $f_i$ has features like:

- Arrival time $t_i$
- Departure time $d_i$
- Aircraft type $a_i$
- Original gate assignment $g_i$

Each gate node $g_j$ has features like:

- Gate type (e.g., wide/narrow body compatibility)
- Availability time window

### 2. **Edge Feature Representation**

For every flight-gate pair $(f_i, g_j)$, I define an edge if:

- The gate is compatible with the aircraft type.
- The gate is free during $[t_i, d_i]$.
- The gate is not occupied by a delayed aircraft.

Edge features can include:

- Time overlap
- Walking distance for passengers
- Gate preference score

### 3. **Graph Update Logic (Message Passing)**

I simulate message passing (simplified GNN behavior):

- Each gate receives "messages" from connected flights representing demand and preference.
- Each flight aggregates "messages" from compatible gates to estimate allocation score.

Let:

- $x_i$ be the feature vector of flight $f_i$
- $x_j$ be the feature vector of gate $g_j$
- $e_{ij}$ be the edge feature between $f_i$ and $g_j$

Then message passing can be defined as:

$$h_{ij} = \text{ReLU}(W_f x_i + W_g x_j + W_e e_{ij})$$

where $W_f$, $W_g$, $W_e$ are learnable (or fixed) weights.

Then:

$s_{ij} = \text{softmax}(h_{ij})$ over all candidate $j$ for a given $i$.

I select the gate $g_j$ with the highest $s_{ij}$.

### 4. **Gate Assignment**

For each delayed flight:

- Remove its old gate (if infeasible).
- Evaluate new gates using updated scores $s_{ij}$.
- Assign the gate $g^*$ that maximizes score while satisfying constraints.

### 5. **Constraints Considered**

- No gate overlap (time conflict).
- Gate-aircraft compatibility.
- Reassign only if delay causes conflict.

### 6. **Output**

- Updated gate assignment list after reallocation.
- List of flights with unresolved gate conflicts (if any).

> **Note**: This version is a simplification. A full GNN model would train weights $W_f$, $W_g$, and $W_e$ on real data with backpropagation and loss minimization (e.g., total passenger walking distance or number of conflicts).
